<a href="https://colab.research.google.com/github/kdpark0284/filtering_titles/blob/main/FilterTitles_Ko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoNLPy Start Up With Google Colab

## Install Dependencies

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

### Install Mecab (Takes 5min+ usually)

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab

In [ ]:
!bash install_mecab-ko_on_colab_light_220429.sh

## (Test before using KoNLPy) -- Not Neccesary

In [ ]:
%%bash
python3 -c "import jpype; jpype.startJVM(convertStrings=True); print(jpype.isJVMStarted())"

In [ ]:
%%bash
pip3 install pytest
TEST_DIR=$(mktemp -d -t test-XXXXXXXXXX)
git clone https://github.com/konlpy/konlpy --depth=1 $TEST_DIR
python3 -m pytest -v $TEST_DIR

## Use KoNLPy Examples

In [ ]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
kkma = Kkma()
komoran = Komoran()
hannanum = Hannanum()
okt = Okt()
pprint(kkma.sentences(u'네, 안녕하세요. 반갑습니다.'))
pprint(komoran.nouns(u'질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))
pprint(hannanum.pos(u'오류보고는 실행환경, 에러메세지와함께 설명을 최대한상세히!^^'))
pprint(okt.phrases(u'구글은 멋져요. 깃허브도 멋져요. KoNLPy도 멋져요!'))

In [ ]:
import konlpy
from konlpy.utils import pprint
from konlpy.tag import Mecab
from konlpy.utils import pprint

mecab = Mecab()

text = "삼성 갤럭시 탭 s8 울트라(6GB+256GB)스페이스블랙[자급제]"
mecabnouns = mecab.nouns(text)
print("Mecab Nouns:", mecabnouns)
mecabmorphs = mecab.morphs(text)
print("Mecab Morphs:", mecabmorphs)


# mecab.tagset

# Extraction Process (Read files, Extract Relevant Words)

## File Upload, Load Dependencies

In [ ]:
import pandas as pd
import openpyxl
from konlpy.tag import Mecab
from konlpy.utils import pprint
from google.colab import files

mecab = Mecab()
df = files.upload ()

"""
Excel 파일 업로드를 해주세요.

"""

df = pd.read_xlsx('HOMEAPPLIANCE_COLLECTION.xlsx')

"""
Excel 파일 이름과 '일치'하게 해주세요.
예시)
test.xlsx 를 업로드한다면 "df = pd.read_xlsx('test.xlsx')"
test.csv 를 업로드한다면 "df = pd.read_csv('test.csv')"
"""


## Product Title


In [ ]:
product_title = {
                '아이폰', '갤럭시', '프로', '울트라', '맥스', '아이패드', '탭', '에어팟', '플립', '폴드', '뮤패드', 'FE'
                '북', '맥북', 'S', 'Ultra', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'Pro', 'ProMax',
                'Galaxy', 'iPhone', 'iPad', 'MacBook', 'Zenbook', '그램'
                  }

product_type_dict = {
    "PC" : "IT",
    "모바일" : "IT",
    "태블릿" : "IT",
    "노트북" : "IT",
    "데스크탑" : "IT",
    "NAFS" : "IT",
    "백색" : "ETC",
    "A/V" : "ETC"
}

general_stopwords_list = [ '에듀몰', '해피투게딜', '자급제', '재고보유', '하이마트배송!', 'WQXGA +', '30만원대',
                        '하이마트배송', '스태킹키트 포함', '스태킹키트 미포함', '한정판매', '한정수량', '온라인 전용', '1주이상소요/클리어런스', '스태킹/앵글',
                        '셀프관리형', '신모델', '즉시배송', '신상', '앵글설치포함,별도비용발생', '하이마트 설치', '최대36개월무', '사은품 증정', '제주마지막1대', '직렬설치',
                        '추가 다운로드 쿠폰', '국내정발', '빠른출발', '진열', '맞춤출수', '전시', '주연테크x하이메이드', '사내판매', '스태킹키트', '카드 추가',
                        '본품', '배송지연', '6개월주기 방문관리형', '익일배송', 'UP가전', '특가', '뭉치면싸다', '1만 다운로드 쿠폰', '1주차', '발송예정', '스태킹설치',
                        '사용안함', '스마트픽',  '미디어', '인버터', '정품OS', '단순배송', '2만원할인', '100원딜', '10대', '한정', '단독!', '단독', '추가 카드',
                        'AppleCare+', '1주이상소요', '**',  'DDR5', 'DDR4', 'DDR6', '시력보호', '최신형', '운영', '스태이션 포함!', '추가 쿠폰', '스태킹', '앵글',
                        '최종321만', '10대 한정판매',  '출고', '안드로이드', '배송시간 소요', '방문관리', 'i5.*?', 'i7.*?', 'win11', '20만원대', '링크!', '키트',
                        '사은품 증정', '2주이상 소요', '동시구매행사용', '1만원할인', '할인', '신학기특가', '키트포함', '키트 포함', 'CPU', 'GPU', 'RTX', 'GTX',
                        '각도조절형', '해피투게딜이벤트', '서랍형', '36개월케어십포함', '개별구매불가,본체만구매-자동취소', '전국무료설치', '8코어', '12코어', '16코어', '10코어',
                        '한정수량', '단종', '소진', '특가', '포토상품평이벤트', '어메이징','3월', '4월', '5월', '6월', '7월', '2주이상소요', '3주이상소요', 'NEW', '가벼운',
                        '8월', '9월', '10월', '11월', '12월', '1월', '2월', '행사', '1등급', '당질저감', '벽걸이조절형', '3년케어십', '초고온수', '기획전', '미포함', '포함',
                        '음성인식', '청정바람', '자가관리', '셀프형', '6개월', '방문주기', '시크릿 !', '최대', '10%', '신모델', '공동구매', '하이마트', '1만 다운로드 쿠폰',
                        '3개월', '창립기획', '단독모델', '스태킹설치포함', '자동먼지비움', '무빙키트' , '공기살균', '쇼핑몰', '상의5벌+하의1벌', '배송!', '외장', '외장그래픽',
                        '전자동', '오프라인', '온라인', '전용', '6인용', '3인용', '개별구매불가 본체만구매-자동취소', '미세먼지극복', '병렬설치', '비밀', '앵글설치포함']

## Definition List

In [ ]:
# Data cleaning Def
def cleaning_general(descriptions, stopwords):
    cleaned_descriptions = []
    for stopword in stopwords:
      cleaned_description = cleaned_description.replace(stopword, '')

    cleaned_descriptions.append(cleaned_description.strip())

    return cleaned_descriptions

# Morphs Extraction Def
def extract_morphs(text):
    morphs = mecab.morphs(text)
    return ' '.join(mecabmorphs)

# Title Decision
def decision_title(morphs,title_list):
  title = ''
  for i, word in morphs:
      if word in title_list:
          title += f'{word}'
          if morphs[i + 1].isdigit():
              title += f'{morphs[i + 1]}'
  return title.strip

# Storage/Ram Decision
def isdigitstorage(morphs):
  storage = 'N/A'
  ram = 'N/A'
  for word in morphs:
    if word is 128 or 256 or 512 or 1024:
      storage = f'{word}GB'
    elif word is 8 or 16 or 32 or 64:
      ram = f'{word}GB'
      return ram, storage
  return False

# Attribute 사전 단어 콜 함수
def get_product_type(attributes, product_type_dict):
    for key in product_type_dict.keys():
        if key in attributes:
            return product_type_dict[key]
    return 'N/A'

